In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mai-ml-2019-linear-classification/test.csv
/kaggle/input/mai-ml-2019-linear-classification/train.csv
/kaggle/input/mai-ml-2019-linear-classification/sample_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/mai-ml-2019-linear-classification/train.csv')

In [3]:
train_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
1,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,0
2,1240,2,"Giles, Mr. Ralph",male,24.0,0,0,248726,13.5000,NaN,S,0
3,221,3,"Sunderland, Mr. Victor Francis",male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S,1
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S,0


In [4]:
train_data.Embarked.fillna("NA")

0      S
1      S
2      S
3      S
4      S
      ..
845    S
846    C
847    S
848    S
849    S
Name: Embarked, Length: 850, dtype: object

In [5]:
def countCabin(cabin):
    if cabin == 'NA':
        return 0
    else:
        return len(cabin.split(' '))

In [6]:
def cal_emb(x):
    if x =='NA':
        return 0
    else:
        return ord(x[0]) - ord('S') +1

In [7]:

def correct_data(train_data,mode):
    train_data['Family'] = train_data.SibSp + train_data.Parch
    train_data.Age = train_data.Age.fillna(train_data.Age.mean())
    train_data.Fare = train_data.Fare.fillna(train_data.Fare.mean())
    train_data['Sex'] = LabelEncoder().fit_transform(train_data['Sex'])
    train_data.Embarked =  train_data.Embarked.fillna("NA")
    train_data.Cabin =  train_data.Cabin.fillna('NA')
    train_data['Alone'] = train_data["Family"] == 0
    train_data.Fare = (train_data.Fare - train_data.Fare.min())/(train_data.Fare.max() - train_data.Fare.min())
    train_data.Age = (train_data.Age - train_data.Age.min())/(train_data.Age.max() - train_data.Age.min())
    train_data.Cabin = train_data.Cabin.apply(countCabin)
    train_data.Embarked = train_data.Embarked.apply(cal_emb)
    train_data.drop(['PassengerId','Name','SibSp','Parch', 'Ticket'], axis=1, inplace=True)
    

    return train_data

In [8]:
train_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,888,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.0000,B42,S,1
1,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,0
2,1240,2,"Giles, Mr. Ralph",male,24.00,0,0,248726,13.5000,NaN,S,0
3,221,3,"Sunderland, Mr. Victor Francis",male,16.00,0,0,SOTON/OQ 392089,8.0500,NaN,S,1
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.00,1,0,24065,26.0000,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
845,756,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.5000,NaN,S,1
846,1297,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.00,0,0,SC/PARIS 2166,13.8625,D38,C,1
847,531,2,"Quick, Miss. Phyllis May",female,2.00,1,1,26360,26.0000,NaN,S,1
848,721,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.00,0,1,248727,33.0000,NaN,S,1


In [9]:
y= train_data['Survived'].values
mode = train_data.Embarked.mode()[0]
train_data = correct_data(train_data,mode)
train_data

,Pclass,Sex,Age,Fare,Cabin,Embarked,Survived,Family,Alone
0,1,0,0.234342,0.058556,1,1,1,0,True
1,3,1,0.370536,0.015379,0,1,0,0,True
2,2,1,0.297101,0.026350,0,1,0,0,True
3,3,1,0.196686,0.015713,0,1,1,0,True
4,2,0,0.748964,0.050749,0,1,0,1,False
...,...,...,...,...,...,...,...,...,...
845,2,1,0.004268,0.028302,0,1,1,2,False
846,2,1,0.246893,0.027058,1,-15,1,0,True
847,2,0,0.020961,0.050749,0,1,1,2,False
848,2,0,0.071169,0.064412,0,1,1,1,False


In [10]:
train_data.drop(['Survived',], axis=1, inplace=True)
   
X = train_data.values
X.shape

(850, 8)

In [11]:

X = train_data.values
X.shape
(850, 8)
C_values = np.arange(1,10000,10)
best_score = 0
best_param = 0.1
best_model = None
early_stopind = 100
round_without_gains = 0
for C in C_values:
    if round_without_gains  == early_stopind:
        break
    model = LogisticRegression(C = C,random_state=48)
    score = np.mean(cross_val_score(model,X,y,cv=5,scoring = 'accuracy'))
    round_without_gains += 1 
    if score > best_score:
        best_model = model
        best_score = score
        best_param = C
        round_without_gains = 0

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [12]:
best_score


0.7905882352941176

In [13]:
best_model.fit(X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=48, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
test_data = pd.read_csv('/kaggle/input/mai-ml-2019-linear-classification/test.csv')
ps_id = test_data.PassengerId.values

In [15]:
test_data_cor = correct_data(test_data,mode)
col_train = train_data.columns
col_test = test_data_cor.columns
for c in col_train:
    if not c in col_test:
        test_data_cor[c] = 0
        
for c in col_test:
    if not c in col_train:
        test_data_cor.drop([c],axis = 1,inplace =True)
        

In [16]:
X = test_data_cor.values
X.shape

(459, 8)

In [17]:
y = best_model.predict(X)

In [18]:
answer = pd.DataFrame({
    'PassengerId': ps_id, 
    'Survived':y,
   },
)

In [19]:
answer.to_csv('output.csv',index=False)